# 1. Amazon Bedrock Knowledge Bases - RAG 생성
- Amazon SageMaker AI - Notebook - JupyterLab 환경에서 테스트 되었습니다.
- Kernel : conda_python3

## 1. Setup

### 필요한 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치 (설치 후 커널 재부팅 필요)
!pip install -q boto3 --upgrade
!pip install -q awscli --upgrade

### 라이브러리 임포트 및 세션 설정

In [ ]:
# 필요한 라이브러리 임포트
import boto3
import json
import time
import os
import uuid
from datetime import datetime
import pandas as pd
from tqdm.notebook import tqdm

# AWS 리전 설정
region = boto3.session.Session().region_name
print(f"현재 AWS 리전: {region}")

# 세션 및 클라이언트 설정
session = boto3.session.Session(region_name=region)
bedrock = session.client('bedrock')
bedrock_runtime = session.client('bedrock-runtime')
bedrock_agent = session.client('bedrock-agent')
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')
s3 = session.client('s3')

In [ ]:
print(boto3.__version__)

### Workshop을 위한 S3 버킷 생성

In [ ]:
# S3 버킷 설정
bucket_name = f"bedrock-kb-workshop-{str(uuid.uuid4())[:8]}"
prefix = "advanced-rag-workshop"

# 버킷이 존재하지 않으면 생성
try:
    s3.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={'LocationConstraint': region}
    )
    print(f"버킷 생성 완료: {bucket_name}")
except Exception as e:
    print(f"버킷 생성 중 오류 발생: {e}")

# S3 경로 설정
s3_data_path = f"s3://{bucket_name}/{prefix}/data"
print(f"데이터 저장 경로: {s3_data_path}")

### S3 버킷에 데이터 (10-q) 업로드

In [ ]:
# ../data/10-q/ 디렉토리의 PDF 파일들을 S3에 업로드
import os
import glob
from tqdm.notebook import tqdm

# PDF 파일 경로 설정
pdf_directory = "./data/10-q/"

# 디렉토리 존재 확인
if not os.path.exists(pdf_directory):
    print(f"경고: {pdf_directory} 디렉토리가 존재하지 않습니다.")
    # 디렉토리 생성 (선택적)
    os.makedirs(pdf_directory, exist_ok=True)
    print(f"{pdf_directory} 디렉토리를 생성했습니다. 이 디렉토리에 PDF 파일을 넣어주세요.")
else:
    # PDF 파일 목록 가져오기
    pdf_files = glob.glob(os.path.join(pdf_directory, "*.pdf"))
    
    if not pdf_files:
        print(f"경고: {pdf_directory} 디렉토리에 PDF 파일이 없습니다.")
    else:
        print(f"{len(pdf_files)}개의 PDF 파일을 찾았습니다. S3에 업로드를 시작합니다...")
        
        # 각 PDF 파일을 S3에 업로드
        for pdf_file in tqdm(pdf_files, desc="PDF 파일 업로드 중"):
            file_name = os.path.basename(pdf_file)
            s3_key = f"{prefix}/data/{file_name}"
            
            try:
                s3.upload_file(
                    Filename=pdf_file,
                    Bucket=bucket_name,
                    Key=s3_key
                )
                print(f"모든 PDF 파일이 s3://{bucket_name}/{prefix}/data/ 경로에 업로드되었습니다.")

            except Exception as e:
                print(f"파일 '{file_name}' 업로드 중 오류 발생: {e}")
        


## 2. KnowledgeBase - RAG 생성 

<b> Amazon Bedrock 콘솔 화면에서 Knowledge Bases 를 선택 합니다. </b>

![graphrag01](../img/graphrag-01.png)

<b> Create 버튼을 누르고 Knowledge Base with vector store를 선택합니다. </b>

![aosrag02](../img/aosrag-02.jpg)

<b> 다음과 같이 정보를 입력 및 선택 합니다. 
- Knowledge Base name : aosrag-workshop
- IAM permission : Create and use a new service role 선택
- Choose data source : Amazon S3 선택
그리고 Next 버튼을 누릅니다.
</b>

![aosrag03](../img/aosrag-03.jpg)

<b> 다음과 같이 정보를 입력 및 선택 합니다. 
- Data source name : 10-q
- S3 URI : <앞서 생성된 S3 경로 s3://~~~/data/> 를 입력 또는 선택 합니다.
그리고 Next 버튼을 누릅니다.
</b>

![aosrag04](../img/aosrag-04.jpg)

임베딩 모델로 Titan Text Embeddings V2를 선택합니다.
![aosrag05](../img/aosrag-05.jpg)

Vector store로 OpenSearch Serverless를 선택한 후 Next를 클릭합니다.
![aosrag06](../img/aosrag-06.jpg)

KB 생성 정보를 확인 후, Create Knowledge Base 버튼을 누릅니다.
![aosrag07](../img/aosrag-07.jpg)

수 분후 KB - OpenSearch Vector Store가 생성되며, 여기서 다음 정보를 확인합니다.
<br>해당 정보는 다음 단계 수행을 위해 값을 복사합니다.
- Knowledge Base ID
![aosrag08](../img/aosrag-08.jpg)

Data source에서 Data Sync를 수행합니다.
* Data source(10-q)를 선택합니다.
* Sync 버튼을 누른 뒤 몇 분 기다리면 Data Sync가 완료됩니다. 이후 Status: Available 상태로 표시됩니다.
![aosrag09](../img/aosrag-09.jpg)

## 3. RAG 실행

앞서 복사한 Knowledge Base ID의 값을 아래 변수에 저장합니다.

In [ ]:
kb_id = "KB ID 입력"

In [ ]:
def query_knowledge_base(query_text, model_id="anthropic.claude-3-5-sonnet-20241022-v2:0", max_tokens=1000):
    try:
        # 검색 요청 구성
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={
                'text': query_text
            },
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 3,
                    'overrideSearchType': 'SEMANTIC'                    
                }
            }
        )
        
        # 검색 결과 확인
        retrieved_results = retrieve_response.get('retrievalResults', [])
        
        if not retrieved_results:
            print("검색 결과가 없습니다.")
            return None
        
        # 검색 결과를 컨텍스트로 사용
        context = ""
        for i, result in enumerate(retrieved_results):
            content = result['content']['text']
            source = result.get('location', {}).get('s3Location', {}).get('uri', '알 수 없는 소스')
            score = result.get('score', 0)
            
            context += f"\n\n참고 문서 {i+1} (관련성 점수: {score}):\n{content}\n"
            print(f"검색 결과 {i+1}: 관련성 점수 {score}")
        
        # Claude에 질의 구성
        prompt = f"""
사용자의 질문에 답변해 주세요. 다음 참고 문서의 정보를 활용하세요:

{context}

사용자 질문: {query_text}

답변:
"""
        
        # Claude에 질의 요청 (수정된 부분)
        response = bedrock_runtime.converse(
            modelId=model_id,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ],
            inferenceConfig={
                "maxTokens": max_tokens,
                "temperature": 0.7,
                "topP": 0.9
            }
        )
        
        # 응답 추출 및 반환
        answer = response['output']['message']['content'][0]['text']
        return answer
        
    except Exception as e:
        print(f"쿼리 실행 중 오류 발생: {e}")
        return None

In [ ]:
# 쿼리 예제 실행
test_queries = [
    "Amazon의 총 순매출은 시간이 지남에 따라 어떻게 변화했나요?",
    "Amazon의 운영 비용 증가가 각 사업 부문의 수익성과 전반적인 재무 성과에 어떤 영향을 미쳤으며, 이는 다른 재무 지표들과 어떤 연관성을 보이나요?",
    "Form 1O-Q 사용에 관한 규칙은 무엇인가요?"
]

for query in test_queries:
    print(f"\n질문: {query}")
    answer = query_knowledge_base(query)
    if answer:
        print(f"\n답변:\n{answer}")
    print("-" * 80)